# Kontaktide parseerimine

**Probleem:**
Telefoninumbrite ja emailide parssimine tekstist. 

Antud probleemile on mitu erinevat kontakstist sõltuvat lähenemisviisi, millele on omakorda mitu erinevat lahendust.

Alljärgnev vihik seletab ühe lähenemisviisi mõtteprotsessi.

<hr>
**Telefoninumbrid:**

Alustan telefoninumbritest, kuna võrreldes emailide parssimisega on see arvatavasti keerulisem.

Telefoninumbrite parssimine on keeruline, kuna on olemas sadu erinevaid telefoni numbrite vorme, näiteks:

- +37259596129 (Eesti number koos suunakoodiga)
- +1-555-123-4567 (USA number koos suunakoodiga ning sidekriipsudega)
- +1-465-864-3443 (USA number koos suunakoodiga ning punktidega)
- 59596129 (Eesti number ilma suunakoodita)
- 1182 (Neljakohaline infoliini telefoni number)
- 112 (Kolmekohaline hädaabi number)

Peale erinevate vormide võivad kontakt numbrid ka erinevalt kirjutatud olla:

- +372 5959 6129
- +372 53546329
- 37256568830
- 56 56 555 3

jne.

Telefoninumbrid võivad ka konflikteeruda tavaliste arvudega, näiteks "1182" võib olla telefoninumber, kuid samas midagi nagu "2017" võib olla sama vabalt ka aastaarv.

Emailid on kergem probleem, neis on vähem dispersiooni.



<hr>
Alustuseks, võtan .json faili, mis sisaldab erinevaid suunakoode vastavalt riigile
Väike tükk .json failist:

Antud failis on hästi välja märgitud riigi kahekohaline tähtkood, riigi nimi ning ka suunakood.

Peale seda alustan Pythoni faili:

In [14]:
# Importin vajalikud teegid 

import sys
import os
import re
import json

#Laen .json faili Python dictionary'sse

AREA_CODES = json.load(open('path_to_area_codes.json'))

print(AREA_CODES['countryCodes'][0])

{'country_code': 'AF', 'country_name': 'Afghanistan', 'dialling_code': '+93'}


Nüüd oleks vaja mingeid meetodeid, läbi mille tekstist andmeid leida.

In [23]:
# Loon klassi, nimega ParseContacts

class ParseContacts():
    '''
    A class that takes in an input of area codes and text and parses out contact addresses (emails, phone numbers)
    from the given text.

    Arguments:
    area_codes -- A dict of country names, area codes, and country codes.

    text -- A string of text to parse the contacts from
    '''
    
    # Klassi konstruktor, võtab argumentideks ala koodide informatsiooni dicti ning text stringi
    def __init__(self, area_codes, text):
        self.area_codes = area_codes
        self.text = text
        
        # Leia tekstist numbrid find_numbers_from_text funktsiooniga.
        self.numbers = self.find_numbers_from_text(self.text, area_codes)
        # Leia tekstist emailid find_emails funktsiooniga.
        self.emails = self.find_emails(self.text)
        # Lisa emailid ja numbrid dictionary'sse
        self.contacts = {'Phone_numbers': self.numbers, 'Emails': self.emails }
        
    # Kirjutame nüüd esimese funktsiooni, mis tegeleb numbrite parssimisega:    
    def find_numbers_from_text(self, text, area_codes):
        '''
        Takes in an input string and parses out all the phone numbers it finds.
        Arguments:
        text -- A string of text to parse phone numbers from.
        Returns:
        numbers -- A list containing the numbers found.
        '''
        # Loon tühja listi, kuhu hiljem lisan kõik telefoninumbrid mis tekstist leidsin
        numbers_to_process = []
        
        # Kasutan regex'it, et leida tekstist telefoninumbreid
        pattern = re.compile(r'(?:\b|[\+])\d{1,4}[-. ]?\d{2,4}(?:[-. ]?\d{1,4}\b)+')
        
        # Loopin üle kõikide leitud numbrite tekstis ning eemaldan tühikud ning punktid
        to_strip = str.maketrans("", "", " -.")
        for n in pattern.findall(text):
            numbers_to_process.append(n.translate(to_strip)) 
        
        # Kutsume meetodi find_area_code_matches
        numbers = self.find_area_code_matches(numbers_to_process, area_codes)
        
        return numbers
    
    #Nüüd kirjutame meetodi, mis leiab suunakoodid teatud numbritest ning lisab kõik numbrid dictionary'sse
    @staticmethod
    def find_area_code_matches(numbers, area_codes):
        '''
        Takes in phone numbers and area codes and returns a list of dicts of details about a given number.
        if a given number doesn't find an area code to match it, it just gets added with the extra details being empty.

        Arguments:
        numbers -- A list of phone numbers in strings

        area_codes -- A dict of area codes and their corresponding country and country code.

        Returns:
        number_list -- A list containing dictionaries filled with detailed, formatted, sorted details for
                       each given phone number.
        '''
        
        # Loon tühja listi, kuhu hiljem lisan kõik sisestatud numbrid kindla vormiga.
        number_list = []

        for number in numbers:
            for country in area_codes['countryCodes']:
                pattern = re.compile('\\' + country['dialling_code'])                
                for match in pattern.findall(number):
                    # Lisan sulud ümber suunakoodi igale leitud suunakoodiga paarile
                    bracketed_num = number.replace(match, '(' + match + ')')
                    number_list.append({'number': bracketed_num, 'match': match,
                                        'country': country['country_name'], 'country_code': country['country_code']})
                    # So we could check if the number is already in the list for non-matched numbers.
                    number = bracketed_num
            
            # Kui ei leindud suunakoodi numbrile
            if number not in [x['number'] for x in number_list]:
                number_list.append(
                    {'number': number, 'match': '', 'country': '', 'country_code': ''})
        
        # Sorteeri kahanevalt numbri pikkuse järgi.
        number_list = sorted(
            number_list, key=lambda k: len(k['number']), reverse=True)
                
        return number_list
         
    # Emailide parssimise funktsioon
    @staticmethod
    def find_emails(text):
        '''
        Find all the emails in the text.

        Arguments:
        text -- A string of text containing text to parse emails from.

        Returns:
        emails -- a list containing found emails from the input text.
        '''
        
        # Otsin emaile tekstist regexiga.
        pattern = re.compile(r'[^\s@]+@[^@.]+[.][a-zA-Z]{1,10}')
        emails = pattern.findall(text)

        return emails

In [27]:
TEXT = '''
    You can contact us through these numbers: +372 5959 6129, +372 53546329.
	If you cant reach either of those, try dialing our landline: 65656660
    Alternatively, you can call our friend: 56565550, or ask for help at the info-line: 1188.
    
	The US line is +1-555-123-4567.
    
    If you prefer to contact us via email, you can contact us at andrus.rähnik@mail.ee, as well as емаил-аддрс@яндекс.c, or at मराठी@yahoo.com'''

FoundContacts = ParseContacts(AREA_CODES, TEXT)

[print(i) for i in [x for x in FoundContacts.contacts['Phone_numbers']]]
print(FoundContacts.contacts['Emails'])

{'number': '(+372)59596129', 'match': '+372', 'country': 'Estonia', 'country_code': 'EE'}
{'number': '(+372)53546329', 'match': '+372', 'country': 'Estonia', 'country_code': 'EE'}
{'number': '(+1)5551234567', 'match': '+1', 'country': 'United States', 'country_code': 'US'}
{'number': '65656660', 'match': '', 'country': '', 'country_code': ''}
{'number': '56565550', 'match': '', 'country': '', 'country_code': ''}
{'number': '1188', 'match': '', 'country': '', 'country_code': ''}
['andrus.rähnik@mail.ee', 'емаил-аддрс@яндекс.c', 'मराठी@yahoo.com']


Tekstis olevad numbrid ning emailid on leitud

<hr>

## Muud meetodid

Üleval olev meetod küll töötab, kuid vastavalt vajadustele, on võimalik muuta selle funktsionaalsust.

Võib-olla ei ole soovi otsida näiteks nelja kohalisi numbried, jne.

Samuti sai leitud ka emailid isegi siis, kui emailis olid võõrtähed, või alternatiivsed domeenid.
Kuid ka sedagi saab muuta vastavalt vajadusele. (Näiteks kui on vaja parssida ka emaile, millel polegi TLD'd, näiteks andrus.rähnik@mail, jne)

<hr>
Kontaktid sai leitud regex'i põhjal, kuid see pole ainuke viis, kuidas seda teha.
Näiteks on olemas erinevad teegid telefoninumbrite valideerimiseks/parssimiseks, näiteks:

- https://pypi.python.org/pypi/phonenumbers
- https://github.com/daviddrysdale/python-phonenumbers (Google libphonenumber library)

